# Výpočty foldovacích trajektorií proteinů a korekce jejich potenciálů s využitím Kubernetes a Jupyter Notebooků

## Aleš Křenek, Konference e-Infra CZ, 10.5.2022


# Stále stejná pohádka ...

- Důvodem existence infrastruktury jsou aplikace, které ji používají
- Infrastruktura se má přizpůsobit aplikacím, ne naopak
- Zázraky se nedějí, je třeba držet se reálných možností
- Ukazujeme možnou cestu jak dlouhodobá spolupráce motivovala konkrétní rozvoj infrastruktury


- Tato prezentace je několik vydloubaných třešniček z https://github.com/ljocha/chicken-and-egg

# Aplikační problémy a postupy jejich řešení

- Tzv. problém "slepice a vejce" při urychlování simulace foldingu proteinu -- simulaci umíme vést rychle k cíli, známe-li cestu; proč ale hledat cestu, když ji známe? 
- Korekce potenciálů -- empirické potenciály fungují dobře pro simulace proteinů, nepřesné pro malé molekuly (léčiva apod.), je třeba je korigovat s použitím kvantově chemických výpočtů.



- Experimentální, opakovaně modifikované postupy -- Jupyter Notebook je vhodné prostředí
- Některé kroky vyžadují velké softwarové balíky a větší výpočetní zdroje (paměť, CPU, GPU)
- Alokovat je na celou dobu by bylo nepřijatelné plýtvání

In [ ]:
import anncolvar

import os
import shutil
from contextlib import redirect_stdout
import re

import concurrent.futures

import numpy as np
import math
from scipy.sparse import coo_matrix,save_npz,load_npz

from pyDOE import lhs
from scipy.sparse.csgraph import shortest_path
from sklearn.manifold import MDS

import PeptideBuilder as pb
import Bio.PDB as pdb
import Bio.SeqUtils as sequtil

import mdtraj as md

import matplotlib.pyplot as plt
import nglview as nv

# from xvg import read_xvg


In [ ]:
ncores = 8
ntomp = 2
ntmpi = ncores // ntomp

In [ ]:
workdir="/work/1L2Y"
os.chdir(workdir)

# Běžná práce s notebookem

<img src="start.pdf" width=80% />

In [ ]:
pdbid="1L2Y"
pdbfile=f"{pdbid}.pdb"
m = md.load(pdbfile)
v = nv.show_mdtraj(m)
v.add_representation("licorice")
v

In [ ]:
nsteps = 12
p = pdb.PDBParser()
instruct = p.get_structure('in',pdbfile)
resl = list(map(lambda r: sequtil.seq1(r.get_resname()),instruct.get_residues()))
nres = len(resl)
out='conf%d.pdb'

itrn = 0
np.random.seed(itrn + 123456789)

phi = lhs(nres - 2, nsteps)
psi = lhs(nres - 2, nsteps)
outf = pdb.PDBIO()

for s in range(nsteps):
    first = pb.Geometry.geometry(resl[0])
    struct = pb.initialize_res(first)
    
    for r in range(1,nres-1):
        if resl[r] == 'P':
            pb.add_residue(struct,resl[r])
        else:
            pb.add_residue(struct,resl[r],phi[s][r-1]*360,psi[2][r-1]*360)
            
    pb.add_residue(struct,resl[nres-1])
        
    fn = out % (itrn * nsteps + s + 1)
    outf.set_structure(struct)
    outf.save(fn)

In [ ]:
!ls -lt conf*pdb | head -12

In [ ]:
tr = md.load([ "conf%d.pdb" % i for i in range(1,nsteps+1)])
idx=tr[0].top.select("name CA")
tr.superpose(tr[0],atom_indices=idx)
v=nv.show_mdtraj(tr)
#v.clear()
v.add_representation("licorice")
v


# Spuštění programu v dalším kontejneru

In [ ]:
base='conf5'
gmx=f"/work/gmx-k8s -w {pdbid}"

In [ ]:
!{gmx} pdb2gmx -f {base}.pdb -o {base}.gro -p {base} -i {base} -water spce -ff amber99 -ignh
!{gmx} editconf -f {base}.gro -o {base}-box.gro -d 1.5 -bt cubic
!{gmx} grompp -f minim.mdp -c {base}-box.gro -p {base}.top -o {base}-min.tpr -po {base}-min.mdp -maxwarn 1

In [ ]:
!{gmx} mdrun -v -deffnm {base}-min -ntomp 1 -nb cpu -pme cpu

<img src="gmx-k8s.pdf" width=80%/>

In [ ]:
v = nv.show_mdtraj(md.load("conf5-min.trr",top="conf5-min.gro"))
v.add_representation("licorice")
v

# Náročnější výpočet v notebooku

In [ ]:
def scale(cores):
    !kubectl scale deployment.apps/chicken-and-egg{os.environ.get('K8S_LABEL')}-placeholder --replicas={cores}

In [ ]:
neighs = 5
dims = 2
tr = md.load('landmarks.pdb')

if False:
    scale(ncores)
    nconf = len(tr)

    row=[]
    col=[]
    dat=[]

    for i in range(nconf):
        d = md.rmsd(tr,tr,frame=i)
        d[range(i+1)] = np.inf
        for _ in range(neighs):
            j = np.argmin(d)
            if d[j] < np.inf:
                row.append(i)
                col.append(j)
                dat.append(d[j])
                row.append(j)
                col.append(i)
                dat.append(d[j])
                d[j] = np.inf

    # store results in sparse matrix
    dist = coo_matrix((dat,(row,col)),shape=(nconf,nconf)) 
    scale(0)

    # check sanity
    print("conformations (original dimensions): ", nconf)
    print("non-zero distances: ", dist.getnnz())
    save_npz("dist.npz",dist)

In [ ]:
dist = load_npz("dist.npz")
scale(ncores)
sp = shortest_path(dist,directed=False)
mds = MDS(n_components=dims,dissimilarity='precomputed',n_jobs=ncores)
emb = mds.fit_transform(sp)
scale(0)

<img src="scale.pdf" width=80%/>

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(*emb.transpose(),marker='.')
plt.show()

# Náročnější výpočet v dalším kontejneru

- Podobně prvnímu příkladu -- výpočet ve vedlejším kontejneru
- Lze alokovat "libovolné" zdroje (CPU, GPU, RAM)
- Různé image pro různý software (Gromacs, Orca, Nvidia tensorflow, ...)
- O spuštění se stará plánovač K8s
- Déle trvající úlohy mohou být restartovány (např. se staly obětí scale() někoho jiného)
- Je třeba ukládat checkpointy a umět se restartovat

# Výsledky

In [ ]:
tr2 = md.load_xtc("md-ann-pbc.xtc",top='1L2Y.gro')
idx=tr2[0].top.select("name CA")
tr2.superpose(tr2[0],atom_indices=idx)
v = nv.show_mdtraj(tr2)
#v.add_representation("licorice")
v

In [ ]:

tr1 = md.load_xtc("md-vanilla-pbc.xtc",top='1L2Y.gro')
idx=tr1[0].top.select("name CA")
tr1.superpose(tr1[0],atom_indices=idx)

tr3 = md.load_xtc("md-pcv-pbc.xtc",top='1L2Y.gro')
idx=tr3[0].top.select("name CA")
tr3.superpose(tr3[0],atom_indices=idx)

def allprogress():
    rmsd1 = md.rmsd(tr1,tr1)
    rg1 = md.compute_rg(tr1)
    rmsd2 = md.rmsd(tr2,tr2)
    rg2 = md.compute_rg(tr2)
    rmsd3 = md.rmsd(tr3,tr3)
    rg3 = md.compute_rg(tr3)

    # XXX: same lenth expected
    l = len(rmsd1)
    l8 = l // 8;
    ticks = np.arange(0,l,l8)
    labels = ticks / 100

    plt.rcParams.update({'font.size': 14})
    _,ax = plt.subplots(2,1,figsize=(20,12))
    #plt.subplot(311)
    ax[0].plot(rmsd1)
    ax[0].plot(rmsd2)
    ax[0].plot(rmsd3)
    ax[0].grid()
    ax[0].set_ylabel('RMSD (nm)')
    ax[0].set_xticks(ticks)
    ax[0].set_xticklabels(labels)
    ax[0].legend(['unbiased','ANN','PCV'])
    #plt.subplot(312)
    ax[1].plot(rg1)
    ax[1].plot(rg2)
    ax[1].plot(rg3)
    ax[1].grid()
    ax[1].set_ylabel('R. gyr. (nm)')
    ax[1].set_xticks(ticks)
    ax[1].set_xticklabels(labels)
    #plt.subplot(313)

    plt.savefig('graphs.pdf')
    plt.show()

In [ ]:
allprogress()

# Shrnutí

- Použili jsme progresivní prostředí K8s k řešení reálného problému, splňuje základní nároky
- Zajímavé výsledky v aplikační oblasti


- Dlouhodobě běžící Jupyter notebook s alokací minimálních zdrojů
- Podružné výpočty lze pouštět přímo, běží v dalších kontejnerech, používají jiné image, alokují potřebné zdroje
- Krátkodobě lze "nafouknout" i kapacitu dostupnou přímo notebooku
- Vše je implementováno jako opakovaně použitelná rozšíření základní infrastuktury

# Poděkování

- *Vojtěch Spiwok, Dalibor Tapl, Guglielmo Tedeschi:* zdroj inspirujících problémů
- *Vladimír Višňovský:* implementace aplikací
- *Viktória Spišaková, Lukáš Hejtmánek:* budování infrastruktury K8s
- *Dalibor Klusáček*: netradiční plánování úloh v K8s


- *GAČR GA22-29667S, MŠMT LM2018140:* chléb náš vezdejší a krásné nové stroje